In [ ]:
import tensorflow as tf
import json
import numpy as np
from matplotlib import pyplot as plt
from pathlib import Path
import cv2
import math
import wandb

data_wd = Path('/data') / 'raspi_face_detection'
img_path = data_wd / 'images'
labels_path = data_wd / 'labels'

model_sets_path = data_wd / 'model_sets'
aug_sets_path = data_wd / 'aug_data'

# 1 Weights & Biases Configuration

In [ ]:
run = wandb.init(
    # set the wandb project where this run will be logged
    project='raspi-facial-recognition',

    # track hyperparameters and run metadata
    config={
        'learning_rate': 0.0001,
        'image_cnt': len(list(img_path.glob('*'))),
        'epochs': 50,
        'batch_size': 128,
        'lr_decay_divider': 0.60,
        'dropout_rate': 0.2,
        'classloss_weight': 0.1,
        'loss_delta_coord_multiplier': 1
        
    }
)

config = wandb.config

### 2.2 Limit GPU Memory Growth

In [ ]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
tf.config.list_physical_devices('GPU')

### 5.2 Load Augmented Images to Tensorflow Dataset

In [ ]:
def load_image(x): 
    byte_img = tf.io.read_file(x)
    img = tf.io.decode_jpeg(byte_img)
    return img

In [ ]:
train_images = tf.data.Dataset.list_files(str(aug_sets_path / 'train' / 'images'/ '*.jpg'), shuffle=False)
train_images = train_images.map(load_image)
train_images = train_images.map(lambda x: tf.image.resize(x, (120,120)))
train_images = train_images.map(lambda x: x/255)

In [ ]:
test_images = tf.data.Dataset.list_files(str(aug_sets_path / 'test' / 'images'/ '*.jpg'), shuffle=False)
test_images = test_images.map(load_image)
test_images = test_images.map(lambda x: tf.image.resize(x, (120,120)))
test_images = test_images.map(lambda x: x/255)

In [ ]:
val_images = tf.data.Dataset.list_files(str(aug_sets_path / 'val' / 'images'/ '*.jpg'), shuffle=False)
val_images = val_images.map(load_image)
val_images = val_images.map(lambda x: tf.image.resize(x, (120,120)))
val_images = val_images.map(lambda x: x/255)

In [ ]:
train_images.as_numpy_iterator().next()

# 6. Prepare Labels

### 6.1 Build Label Loading Function

In [ ]:
import itertools

In [ ]:
# fn = '49e246c6-0968-11ef-b027-dca632a68397_5.json' # both
# fn = '3a38b642-0968-11ef-b027-dca632a68397_4.json' # kara
# fn = '4a2f0b44-0965-11ef-a3f3-dca632a68397_37.json' # brendan
fn = '369aab30-0968-11ef-b027-dca632a68397_0.json' # error
label_path = aug_sets_path / 'train' / 'labels' / fn
with open(label_path, 'r', encoding = "utf-8") as f:
    label = json.load(f)

label['classes'], label['bboxes']

In [ ]:
def load_labels(label_path):
    with open(label_path.numpy(), 'r', encoding = "utf-8") as f:
        label = json.load(f)

    return [int(x in label['classes']) for x in range(1, 3)], list(itertools.chain(*label['bboxes']))

# classes, bboxes = load_labels(label_path)

### 6.2 Load Labels to Tensorflow Dataset

In [ ]:
train_labels = tf.data.Dataset.list_files(str(aug_sets_path / 'train' / 'labels'/ '*.json'), shuffle=False)
train_labels = train_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.uint8, tf.float16]))

In [ ]:
test_labels = tf.data.Dataset.list_files(str(aug_sets_path / 'test' / 'labels'/ '*.json'), shuffle=False)
test_labels = test_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.uint8, tf.float16]))

In [ ]:
val_labels = tf.data.Dataset.list_files(str(aug_sets_path / 'val' / 'labels'/ '*.json'), shuffle=False)
val_labels = val_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.uint8, tf.float16]))

In [ ]:
train_labels.as_numpy_iterator().next()

# 7. Combine Label and Image Samples

### 7.1 Check Partition Lengths

In [ ]:
len(train_images), len(train_labels), len(test_images), len(test_labels), len(val_images), len(val_labels)

### 7.2 Create Final Datasets (Images/Labels)

In [ ]:
train = tf.data.Dataset.zip((train_images, train_labels))
train = train.shuffle(len(train_images))
train = train.batch(config.batch_size)
train = train.prefetch(4)

In [ ]:
test = tf.data.Dataset.zip((test_images, test_labels))
test = test.shuffle(len(test_images))
test = test.batch(config.batch_size)
test = test.prefetch(4)

In [ ]:
val = tf.data.Dataset.zip((val_images, val_labels))
val = val.shuffle(len(val_images))
val = val.batch(config.batch_size)
val = val.prefetch(4)

### 7.3 View Images and Annotations

In [ ]:
data_samples = train.as_numpy_iterator()

In [ ]:
res = data_samples.next()
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx in range(4): 
    sample_image = res[0][idx]
    sample_coords = res[1][1][idx]
    sample_labels = ', '.join(res[1][0][idx].astype(str).tolist())

    cv2.rectangle(sample_image, 
                  tuple(np.multiply(sample_coords[0:2], [120,120]).astype(int)),
                  tuple(np.multiply(sample_coords[2:4], [120,120]).astype(int)), (255,0,0), 2)
    cv2.rectangle(sample_image, 
                  tuple(np.multiply(sample_coords[4:6], [120,120]).astype(int)),
                  tuple(np.multiply(sample_coords[6:8], [120,120]).astype(int)), (0,255,0), 2)

    ax[idx].imshow(sample_image)
    ax[idx].set_title(sample_labels)

# 8. Build Deep Learning using the Functional API

### 8.1 Import Layers and Base Network

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Dense, GlobalMaxPooling2D, Dropout
from tensorflow.keras.applications import VGG16

### 8.2 Download VGG16

In [ ]:
vgg = VGG16(include_top=False)

In [ ]:
vgg.summary()

### 8.3 Build instance of Network

In [ ]:
def build_model(): 
    input_layer = Input(shape=(120,120,3))
    
    vgg = VGG16(include_top=False)(input_layer)

    # Classification Model  
    f1 = GlobalMaxPooling2D()(vgg)
    class1 = Dense(2048, activation='relu')(f1)
    class1 = tf.keras.layers.Dropout(config.dropout_rate)(class1)
    class2 = Dense(2, activation='sigmoid')(class1)
    
    # Bounding box model
    f2 = GlobalMaxPooling2D()(vgg)
    regress1 = Dense(2048, activation='relu')(f2)
    regress1 = Dropout(config.dropout_rate)(regress1)
    regress2 = Dense(8, activation='sigmoid')(regress1)
    
    facetracker = Model(inputs=input_layer, outputs=[class2, regress2])
    return facetracker

### 8.4 Test out Neural Network

In [ ]:
facetracker = build_model()

In [ ]:
facetracker.summary()

In [ ]:
X, y = train.as_numpy_iterator().next()

# 9. Define Losses and Optimizers

### 9.1 Define Optimizer and LR

In [ ]:
batches_per_epoch = len(train)
lr_decay = (1./config.lr_decay_divider -1)/batches_per_epoch
lr_decay

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=config.learning_rate, decay=lr_decay)

### 9.2 Create Localization Loss and Classification Loss

In [ ]:
def localization_loss(y_true, yhat):
    # differences between coordinates
    delta_coord = tf.reduce_sum(tf.square(y_true[:,:2] - yhat[:,:2])) + tf.reduce_sum(tf.square(y_true[:,4:6] - yhat[:,4:6]))

    # differences between size
    delta_size = 0
    for i in range(2):
        h_true = y_true[:,3+4*i] - y_true[:,1+4*i] 
        w_true = y_true[:,2+4*i] - y_true[:,0+4*i] 
    
        h_pred = yhat[:,3+4*i] - yhat[:,1+4*i] 
        w_pred = yhat[:,2+4*i] - yhat[:,0+4*i] 
        
        delta_size += tf.reduce_sum(tf.square(w_true - w_pred) + tf.square(h_true-h_pred))
    
    return delta_coord * config.loss_delta_coord_multiplier + delta_size

In [ ]:
classloss = tf.keras.losses.BinaryCrossentropy()
regressloss = localization_loss

# 10. Train Neural Network

### 10.1 Create Custom Model Class

In [ ]:
class FaceTracker(Model): 
    def __init__(self, eyetracker,  **kwargs): 
        super().__init__(**kwargs)
        self.model = eyetracker

    def compile(self, opt, classloss, localizationloss, **kwargs):
        super().compile(**kwargs)
        self.closs = classloss
        self.lloss = localizationloss
        self.opt = opt

    def lossfn(self, y, yhat_classes, yhat_coords):
        batch_classloss = self.closs(y[0], yhat_classes)
        batch_localizationloss = self.lloss(tf.cast(y[1], tf.float32), yhat_coords)
        
        total_loss = batch_localizationloss + config.classloss_weight*batch_classloss
        return total_loss, batch_classloss, batch_localizationloss
    
    def train_step(self, batch, **kwargs): 
        
        X, y = batch
        
        with tf.GradientTape() as tape: 
            yhat_classes, yhat_coords = self.model(X, training=True)

            total_loss, batch_classloss, batch_localizationloss = self.lossfn(y, yhat_classes, yhat_coords)
            
            grad = tape.gradient(total_loss, self.model.trainable_variables)
        
        opt.apply_gradients(zip(grad, self.model.trainable_variables))
        
        return {"total_loss":total_loss, "class_loss":batch_classloss, "regress_loss":batch_localizationloss}
    
    def test_step(self, batch, **kwargs): 
        X, y = batch
        
        yhat_classes, yhat_coords = self.model(X, training=False)
        total_loss, batch_classloss, batch_localizationloss = self.lossfn(y, yhat_classes, yhat_coords)
        
        return {"total_loss":total_loss, "class_loss":batch_classloss, "regress_loss":batch_localizationloss}
        
    def call(self, X, **kwargs): 
        return self.model(X, **kwargs)

model = FaceTracker(facetracker)

model.compile(opt, classloss, regressloss)

### 10.2 Train

In [ ]:
logdir='logs'

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
hist = model.fit(train, epochs=config.epochs, validation_data=val, 
                 callbacks=[tensorboard_callback, wandb.keras.WandbMetricsLogger()])

run.finish()

### 10.3 Plot Performance

In [ ]:
fig, ax = plt.subplots(ncols=3, figsize=(20,5))

ax[0].plot(hist.history['total_loss'], color='teal', label='loss')
ax[0].plot(hist.history['val_total_loss'], color='orange', label='val loss')
ax[0].title.set_text('Loss')
ax[0].legend()

ax[1].plot(hist.history['class_loss'], color='teal', label='class loss')
ax[1].plot(hist.history['val_class_loss'], color='orange', label='val class loss')
ax[1].title.set_text('Classification Loss')
ax[1].legend()

ax[2].plot(hist.history['regress_loss'], color='teal', label='regress loss')
ax[2].plot(hist.history['val_regress_loss'], color='orange', label='val regress loss')
ax[2].title.set_text('Regression Loss')
ax[2].legend()

plt.show()

# 11. Make Predictions

### 11.1 Make Predictions on Test Set

In [ ]:
test_data = val.as_numpy_iterator()
test_sample = test_data.next()

In [ ]:
test_sample = test_data.next()

yhat = facetracker.predict(test_sample[0])

fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx in range(4): 
    sample_image = test_sample[0][idx]
    sample_coords = yhat[1][idx]
    
    if yhat[0][idx][0] > 0.9:
        cv2.rectangle(sample_image, 
                      tuple(np.multiply(sample_coords[0:2], [120,120]).astype(int)),
                      tuple(np.multiply(sample_coords[2:4], [120,120]).astype(int)), (255,0,0), 2)
    if yhat[0][idx][1] > 0.9:
        cv2.rectangle(sample_image, 
                      tuple(np.multiply(sample_coords[4:6], [120,120]).astype(int)),
                      tuple(np.multiply(sample_coords[6:8], [120,120]).astype(int)), (0,255,0), 2)
    
    ax[idx].imshow(sample_image)
    ax[idx].set_title(', '.join([f'{x:,.0%}' for x in yhat[0][idx]]))

### 11.2 Save the Model

In [ ]:
facetracker.save('facetracker.h5')